This project used the pre-trained YOLOv8 model to detect vehicles in the video and a custom trained model to detect the number plates.

EasyOCR is used to recognize the character on the number plate.

SORT is used for object tracking.
sort.py from https://github.com/abewley/sort

Reference: https://github.com/computervisioneng/automatic-number-plate-recognition-python-yolov8/tree/main



### Install Dependencies

In [ ]:
!pip install ultralytics easyocr filterpy lap scikit-image==0.17.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.0/755.0 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 60.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.8/29.8 MB 42.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 29.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
PATH = '/content/drive/MyDrive/mydata/ComputerVision/NumberPlateRecognition/'

## Import Libraries


In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import string
import easyocr
from sort import *

## Read number plates using easyocr

In [ ]:
# Initialize OCR reader to read content of number plate
reader = easyocr.Reader(['en'])

# Dictionaries for character conversion for similar looking character and numbers
dict_char_to_int = {'O': '0',
                    'I': '1',
                    'J': '3',
                    'A': '4',
                    'G': '6',
                    'S': '5'}

dict_int_to_char = {'0': 'O',
                    '1': 'I',
                    '3': 'J',
                    '4': 'A',
                    '6': 'G',
                    '5': 'S'}

# check if the number plate complies to the UK number plate format
def UK_number_plate_format(text):
# first two

  if len(text) != 7:
    return False

  if (text[0] in string.ascii_uppercase or text[0] in dict_int_to_char.keys()) and \
      (text[1] in string.ascii_uppercase or text[1] in dict_int_to_char.keys()) and \
      (text[2] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[2] in dict_char_to_int.keys()) and \
      (text[3] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[3] in dict_char_to_int.keys()) and \
      (text[4] in string.ascii_uppercase or text[4] in dict_int_to_char.keys()) and \
      (text[5] in string.ascii_uppercase or text[5] in dict_int_to_char.keys()) and \
      (text[6] in string.ascii_uppercase or text[6] in dict_int_to_char.keys()):

        return True

  else:
        return False

# conversion between similar character and number to address misidentification by easyocr
def formatting_number_plate(text):
  num_plate = ''

  # For a UK number plate only the 3rd and 4th characters will be numbers, others are all alphabets
  mapping = {0: dict_int_to_char, 1: dict_int_to_char, 4: dict_int_to_char, 5: dict_int_to_char, 6: dict_int_to_char,
            2: dict_char_to_int, 3: dict_char_to_int}

  for idx in [0, 1, 2, 3, 4, 5,6]:
    if text[idx] in mapping[idx].keys():
      num_plate += mapping[idx][text[idx]]

    else:
      num_plate += text[idx]

  return num_plate

# read the content of number plate
def read_number_plate(preprocessed_number_plate):

  read_content = reader.readtext(preprocessed_number_plate)

  for r in read_content:
    bbox, text, score = r

    text = text.upper().replace(' ', '')

    if UK_number_plate_format(text):
      return formatting_number_plate(text), score

  return None, None

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [ ]:
# retrieve vehicle coordinates and ID based on the number plate coordinates
def get_vehicle(number_plate, vehicle_track_ids):
  x1, y1, x2, y2, score, class_id = number_plate

  found_vehicle = False
  for idx in range(len(vehicle_track_ids)):
    veh_x1, veh_y1, veh_x2, veh_y2, veh_id = vehicle_track_ids[idx]

    # if the bounding box of number plate is within the bounding box of vehicle, the vehicle is found
    if x1 > veh_x1 and y1 > veh_y1 and x2 < veh_x2 and y2 < veh_y2:
      vehicle_idx = idx
      found_vehicle = True
      break

  if found_vehicle:
    return vehicle_track_ids[vehicle_idx]

  return -1, -1, -1, -1, -1

## Vehicles and Number Plates Detection using YOLOv8

In [ ]:
# load pretrained model that will be used to identify the vehicles in the video
coco_model = YOLO('yolov8n.pt')

# load customed trained model that will be used to identify number plates in the video
number_plate_detector = YOLO('/content/drive/MyDrive/mydata/ComputerVision/VehicleRegistrationPlate/yolov8_number_plate.pt')

100%|██████████| 6.23M/6.23M [00:00<00:00, 215MB/s]


In [ ]:
# save results to a csv file
def write_csv(results, output_path):

    with open(output_path, 'w') as f:
        f.write('frame_num,vehicle_id,vehicle_bbox,number_plate_bbox,number_plate_bbox_score,number_plate,number_plate_score\n')

        for frame_num, frame_results in results.items():
            for veh_id, veh_data in frame_results.items():
                veh_bbox = veh_data.get('vehicle', {}).get('bbox', [0, 0, 0, 0])
                num_plate_bbox = veh_data.get('number_plate', {}).get('bbox', [0, 0, 0, 0])
                num_plate_text = veh_data.get('number_plate', {}).get('text', '')
                num_plate_bbox_score = veh_data.get('number_plate', {}).get('bbox_score', '')
                num_plate_text_score = veh_data.get('number_plate', {}).get('text_score', '')

                f.write(f"{frame_num},{veh_id},{' '.join(map(str, veh_bbox))},{' '.join(map(str, num_plate_bbox))},{num_plate_bbox_score},{num_plate_text},{num_plate_text_score}\n")
        f.close()

    print("CSV file written successfully.")


In [ ]:
results = {}
motion_tracker = Sort()

# load video data
vid = cv2.VideoCapture(PATH + 'data/highway_vid1.mp4')

# class id for vehicles in the pretrained model {2: 'car', 3: 'motorcycle', 5: 'bus', 7: 'truck'}
vehicles_idx = [2,3,5,7]

# read video frames
frame_num = -1
ret = True
while ret:
  frame_num += 1
  ret, frame = vid.read()
  if ret:
    results[frame_num] = {}

    # detect objects in the video
    detections = coco_model(frame)[0]
    vehicles_detected = []

    for det in detections.boxes.data.tolist():
      # coordinates of bounding boxes, confidence score of identified object, class id of identifed object
      x1, y1, x2, y2, score, class_id = det

      # only append objects that are identified as vehicles
      if int(class_id) in vehicles_idx:
        vehicles_detected.append([x1, y1, x2, y2, score])

    # track vehicles in the video
    track_ids = motion_tracker.update(np.asarray(vehicles_detected))

    # detect number plates
    number_plates = number_plate_detector(frame)[0]
    for num_plate in number_plates.boxes.data.tolist():
      x1, y1, x2, y2, score, class_id = num_plate

      # assign number plate to its correscponding vehicle
      veh_x1, veh_y1, veh_x2, veh_y2, veh_id = get_vehicle(num_plate, track_ids)

      # if vehicle exists
      if veh_id != -1:

        # crop number plate
        num_plate_crop = frame[int(y1):int(y2), int(x1): int(x2), :]
        # convert to gray scale
        num_plate_crop_gray = cv2.cvtColor(num_plate_crop, cv2.COLOR_BGR2GRAY)
        # convert the number plate text to white and background to black using binary inverted threshold
        _, num_plate_BW = cv2.threshold(num_plate_crop_gray, 64,255,cv2.THRESH_BINARY_INV)

        # read content of number plate
        num_plate_text, num_plate_text_score = read_number_plate(num_plate_BW)

        if num_plate_text is not None:
          results[frame_num][veh_id] = {'vehicle': {'bbox': [veh_x1, veh_y1, veh_x2, veh_y2]},
                                        'number_plate': {'bbox': [x1, y1, x2, y2],
                                                          'text': num_plate_text,
                                                          'bbox_score': score,
                                                          'text_score': num_plate_text_score}}


# save results to a csv file
write_csv(results, PATH+'highway_vid1_results.csv')

Streaming output truncated to the last 5000 lines.
Speed: 4.0ms preprocess, 156.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 15 cars, 2 buss, 2 trucks, 162.3ms
Speed: 8.3ms preprocess, 162.3ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 vehicle plate, 168.7ms
Speed: 4.5ms preprocess, 168.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 16 cars, 2 buss, 1 truck, 198.7ms
Speed: 5.2ms preprocess, 198.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 vehicle plate, 162.6ms
Speed: 9.0ms preprocess, 162.6ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 16 cars, 1 bus, 1 truck, 161.8ms
Speed: 10.2ms preprocess, 161.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 vehicle plate, 180.8ms
Speed: 8.1ms preprocess, 180.8ms inference, 1.0ms postprocess per image at shap

## Visualize results

In [ ]:
import csv
import numpy as np
import ast
import pandas as pd

In [ ]:
def draw_border(img, top_left, bottom_right, color=(0, 255, 0), thickness=10, line_length_x=200, line_length_y=200):
    x1, y1 = top_left
    x2, y2 = bottom_right

    cv2.line(img, (x1, y1), (x1, y1 + line_length_y), color, thickness)  #-- top-left
    cv2.line(img, (x1, y1), (x1 + line_length_x, y1), color, thickness)

    cv2.line(img, (x1, y2), (x1, y2 - line_length_y), color, thickness)  #-- bottom-left
    cv2.line(img, (x1, y2), (x1 + line_length_x, y2), color, thickness)

    cv2.line(img, (x2, y1), (x2 - line_length_x, y1), color, thickness)  #-- top-right
    cv2.line(img, (x2, y1), (x2, y1 + line_length_y), color, thickness)

    cv2.line(img, (x2, y2), (x2, y2 - line_length_y), color, thickness)  #-- bottom-right
    cv2.line(img, (x2, y2), (x2 - line_length_x, y2), color, thickness)

    return img

In [ ]:
results = pd.read_csv(PATH+'highway_vid1_results.csv')

# load video
vid = cv2.VideoCapture(PATH+'data/highway_vid1.mp4')

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Specify the codec
fps = vid.get(cv2.CAP_PROP_FPS)
width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(PATH+'output_result.mp4', fourcc, fps, (width, height))

number_plate = {}
for vehicle_id in np.unique(results['vehicle_id']):
    max_ = np.amax(results[results['vehicle_id'] == vehicle_id]['number_plate_score'])
    number_plate[vehicle_id] = {'number_crop': None,
                                'number_plate_number': results[(results['vehicle_id'] == vehicle_id) &
                                                                (results['number_plate_score'] == max_)]['number_plate'].iloc[0]}
    vid.set(cv2.CAP_PROP_POS_FRAMES, results[(results['vehicle_id'] == vehicle_id) &
                                             (results['number_plate_score'] == max_)]['frame_num'].iloc[0])
    ret, frame = vid.read()

    x1, y1, x2, y2 = ast.literal_eval(results[(results['vehicle_id'] == vehicle_id) &
                                              (results['number_plate_score'] == max_)]['number_plate_bbox'].iloc[0].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))

    number_crop = frame[int(y1):int(y2), int(x1):int(x2), :]
    number_crop = cv2.resize(number_crop, (int((x2 - x1) * 400 / (y2 - y1)), 400))

    number_plate[vehicle_id]['number_crop'] = number_crop


frame_num = -1

vid.set(cv2.CAP_PROP_POS_FRAMES, 0)
# read frames
ret = True
while ret:
    ret, frame = vid.read()
    frame_num += 1
    if ret:
        df_ = results[results['frame_num'] == frame_num]
        # Clear previous drawings
        frame_with_drawings = np.copy(frame)
        for row_indx in range(len(df_)):
            # Check if a vehicle is detected in the current frame
            if not df_.empty:
                # draw vehicle
                vehicle_x1, vehicle_y1, vehicle_x2, vehicle_y2 = ast.literal_eval(df_.iloc[row_indx]['vehicle_bbox'].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))
                draw_border(frame_with_drawings, (int(vehicle_x1), int(vehicle_y1)), (int(vehicle_x2), int(vehicle_y2)), (0, 255, 0), 25,
                            line_length_x=200, line_length_y=200)

                # draw number plate
                x1, y1, x2, y2 = ast.literal_eval(df_.iloc[row_indx]['number_plate_bbox'].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))
                cv2.rectangle(frame_with_drawings, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 12)

                # crop number plate
                number_crop = number_plate[df_.iloc[row_indx]['vehicle_id']]['number_crop']

                H, W, _ = number_crop.shape

                try:
                    frame_with_drawings[int(vehicle_y1) - H - 100:int(vehicle_y1) - 100,
                          int((vehicle_x2 + vehicle_x1 - W) / 2):int((vehicle_x2 + vehicle_x1 + W) / 2), :] = number_crop

                    frame_with_drawings[int(vehicle_y1) - H - 400:int(vehicle_y1) - H - 100,
                          int((vehicle_x2 + vehicle_x1 - W) / 2):int((vehicle_x2 + vehicle_x1 + W) / 2), :] = (255, 255, 255)

                    (text_width, text_height), _ = cv2.getTextSize(
                        number_plate[df_.iloc[row_indx]['vehicle_id']]['number_plate_number'],
                        cv2.FONT_HERSHEY_SIMPLEX,
                        4.3,
                        17)

                    cv2.putText(frame_with_drawings,
                                number_plate[df_.iloc[row_indx]['vehicle_id']]['number_plate_number'],
                                (int((vehicle_x2 + vehicle_x1 - text_width) / 2), int(vehicle_y1 - H - 250 + (text_height / 2))),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                4.3,
                                (0, 0, 0),
                                17)

                except:
                    pass
        out.write(frame_with_drawings)
        frame_with_drawings = cv2.resize(frame_with_drawings, (1280, 720))
out.release()
vid.release()




In [ ]:
# read frames
ret = True
while ret:
    ret, frame = vid.read()
    frame_num += 1
    if ret:
        df_ = results[results['frame_num'] == frame_num]
        for row_indx in range(len(df_)):
            # draw vehicle
            vehicle_x1, vehicle_y1, vehicle_x2, vehicle_y2 = ast.literal_eval(df_.iloc[row_indx]['vehicle_bbox'].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))
            draw_border(frame, (int(vehicle_x1), int(vehicle_y1)), (int(vehicle_x2), int(vehicle_y2)), (0, 255, 0), 25,
                        line_length_x=200, line_length_y=200)

            # draw number plate
            x1, y1, x2, y2 = ast.literal_eval(df_.iloc[row_indx]['number_plate_bbox'].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 12)

            # crop number plate
            number_crop = number_plate[df_.iloc[row_indx]['vehicle_id']]['number_crop']

            H, W, _ = number_crop.shape

            try:
                frame[int(vehicle_y1) - H - 100:int(vehicle_y1) - 100,
                      int((vehicle_x2 + vehicle_x1 - W) / 2):int((vehicle_x2 + vehicle_x1 + W) / 2), :] = number_crop

                frame[int(vehicle_y1) - H - 400:int(vehicle_y1) - H - 100,
                      int((vehicle_x2 + vehicle_x1 - W) / 2):int((vehicle_x2 + vehicle_x1 + W) / 2), :] = (255, 255, 255)

                (text_width, text_height), _ = cv2.getTextSize(
                    number_plate[df_.iloc[row_indx]['vehicle_id']]['number_plate_number'],
                    cv2.FONT_HERSHEY_SIMPLEX,
                    4.3,
                    17)

                cv2.putText(frame,
                            number_plate[df_.iloc[row_indx]['vehicle_id']]['number_plate_number'],
                            (int((vehicle_x2 + vehicle_x1 - text_width) / 2), int(vehicle_y1 - H - 250 + (text_height / 2))),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            4.3,
                            (0, 0, 0),
                            17)

            except:
                pass

        out.write(frame)
        frame = cv2.resize(frame, (1280, 720))

out.release()
vid.release()